In [1]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster
from folium import plugins
from folium.map import FeatureGroup
import os

In [2]:
crimes = pd.read_csv('data/crimes.csv')

In [3]:
download_maps = True

## All crimes

In [4]:
if download_maps and not os.path.exists('maps'):
    os.makedirs('maps')

In [22]:
def generateBaseMap(default_location=[-22.010147, -47.890706], default_zoom_start=13):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

callback = ('function (row) {'
                'var circle = L.circle(new L.LatLng(row[0], row[1]));'
                'return circle};')

In [6]:
crimes_clusters_map = generateBaseMap()

points = crimes[['LATITUDE', 'LONGITUDE']].values.tolist()
crimes_clusters_map.add_child(FastMarkerCluster(points, callback=callback))
crimes_clusters_map

## Veículos

In [7]:
crimes['Conduta'].unique()

array(['VEICULO', 'OUTROS', 'TRANSEUNTE', 'ESTABELECIMENTO COMERCIAL',
       'CONDOMINIO COMERCIAL', 'ESTABELECIMENTO ENSINO',
       'ESTABELECIMENTO-OUTROS', 'INTERIOR ESTABELECIMENTO', nan,
       'INTERIOR DE VEICULO', 'COLETIVO', 'INTERIOR TRANSPORTE COLETIVO',
       'SAIDINHA DE BANCO', 'CAIXA ELETRÔNICO'], dtype=object)

In [8]:
crimes['Conduta'].value_counts()

VEICULO                         2725
TRANSEUNTE                      1802
ESTABELECIMENTO COMERCIAL        931
OUTROS                           365
ESTABELECIMENTO-OUTROS            75
COLETIVO                          72
INTERIOR TRANSPORTE COLETIVO      67
INTERIOR DE VEICULO               64
INTERIOR ESTABELECIMENTO          59
ESTABELECIMENTO ENSINO             7
SAIDINHA DE BANCO                  3
CONDOMINIO COMERCIAL               2
CAIXA ELETRÔNICO                   1
Name: Conduta, dtype: int64

In [24]:
all_with_layers = generateBaseMap() 

fg = FeatureGroup(name='Todos os crimes')
points = crimes[['LATITUDE', 'LONGITUDE']].values.tolist()
FastMarkerCluster(points, callback=callback).add_to(fg)
fg.add_to(all_with_layers)

tipos = [tipo for tipo in crimes['Conduta'].unique() if isinstance(tipo, str)]

for tipo in sorted(tipos):
    # adding new layer
    fg = FeatureGroup(name=tipo.capitalize())
    curr = generateBaseMap()
    points = crimes[crimes['Conduta'] == tipo][['LATITUDE', 'LONGITUDE']].values.tolist()    
    FastMarkerCluster(points, callback=callback, show=False).add_to(fg)
    fg.add_to(all_with_layers)

    # creating different map
    if download_maps:
        new_map = generateBaseMap()
        new_map.add_child(FastMarkerCluster(points, callback=callback))
        new_map.save(outfile=f'maps/{tipo.lower()}_clusters.html')
    
    
folium.LayerControl().add_to(all_with_layers)
all_with_layers

In [10]:
if download_maps:
    all_with_layers.save(outfile=f'maps/all_with_layers_clusters.html')